# ECON408: Computational Methods in Macroeconomics

Search Models

Jesse Perla (University of British Columbia)

# Overview

## Motivation

-   In the previous lecture we described a model with employment and
    unemployment as a Markov Chain
-   Central to that were arrival rates of transitions between the $U$
    and $E$ states at some $\lambda$ probability each period.
-   But the worker didn’t have a choice whether to accept the job or
    not. In this lecture we will investigate a simple model where
    workers search for jobs, and the $\lambda$ becomes an endogenous
    choice
-   As with the previous lecture on the [Permanent Income
    model](https://jlperla.github.io/ECON408/lectures/permanent_income.html#/exogenous-vs.-endogenous-savings),
    the benefit is that we can consider policy counterfactuals which may
    affect the workers choices

## Materials

-   Adapted from QuantEcon lectures coauthored with John Stachurski and
    Thomas J. Sargent
    -   [Job Search I: The McCall Search
        Model](https://julia.quantecon.org/dynamic_programming/mccall_model.html)
    -   [Job Search II: Search and
        Separation](https://julia.quantecon.org/dynamic_programming/mccall_model_with_separation.html)
    -   [A Lake Model of Employment and
        Unemployment](https://julia.quantecon.org/multi_agent_models/lake_model.html)

In [1]:
using LinearAlgebra, Statistics
using Distributions, LaTeXStrings
using Plots.PlotMeasures, NLsolve, Roots, Random, Plots
default(;legendfontsize=16, linewidth=2, tickfontsize=12,
         bottom_margin=15mm)

# The McCall Model

## Summary

-   See
    [here](https://julia.quantecon.org/dynamic_programming/mccall_model.html)
    for a more minimal verison
-   The McCall model is a model of “search” in the labor market
-   A worker can be in a state of unemployment or employment. at wage
    $W_t$
-   The key decision: if they are unemployed and receive a job offer at
    wage $W_t$, should they accept it or keep searching?
-   Assume that wages come from a fixed, known distribution with
    discrete values $w' \in \{w_1, \ldots, w_N\}$ and probabilities
    $\{p_1, \ldots, p_N\}$

## Preferences

-   Let $Y_t$ be the stochastic payoffs of the consumer

    -   $Y_t = W_t$ if employed at wage $W_t$
    -   $Y_t = c$, unemployment insurance, while unemployed

-   Let $u(\cdot)$ be a standard utility function with $u'(\cdot) > 0$
    and $u''(\cdot) \leq 0$

-   Preferences over stochastic incomes $\{Y_{t+j}\}_{j=0}^\infty$ given
    time $t$ information are

    $$
    \mathbb{E}_t \sum_{j=0}^\infty \beta^j u(Y_{t+j})
    $$

## Timeline and Decisions

-   If they are employed at wage $w$ they
    -   get the wage $w$ and enter the next period.
    -   have some probability $\alpha$ the job ends and they become
        unemployed before next period starts, otherwise the $w$ does not
        change
-   If they are unemployed they
    -   get unemployment insurance, $c$
    -   have some probability, $\gamma$ of getting a wage offer, $w'$
        for the next period.
    -   Given some wage offer, $w'$, they can choose to accept the job
        and enter employment, or to reject the offer and remain
        unemployed
    -   Cannot recall previously rejected wages (but wouldn’t, in
        equilibrium)

## A Trade Off

-   The worker faces a trade-off:

    -   Waiting too long for a good offer is costly, since the future is
        discounted.
    -   Accepting too early is costly, since better offers might arrive
        in the future.

-   To decide optimally in the face of this trade off, we use dynamic
    programming

-   The key is to start with the state:

    -   employment status
    -   the current wage, $w$, if employed

-   Then determine the feasible set of actions, and how the state
    evolves under each action

## Value Functions

-   All unemployment states are the same since nothing is changing over
    time and the previous wages are irrelevant
    -   Hence, let $U$ be the value of being unemployed
-   The only thing that matters for the employed worker is the current
    wage, $w$
    -   Hence, let $V(w)$ be the value of being employed at wage $w$

## Value Function While Employed

-   The employed agent is passive
-   While employed, at the end of the period
    -   With some probability $\alpha$ they become unemployed with value
        $U$
    -   Otherwise, they remain employed with value $V(w)$

## Actions and Transitions with Unemployed

-   If unemployed,
    -   With some probably $\gamma$ they get a wage offer $w'$
    -   If they **accept** they enter employment at wage $w'$ next
        period (i.e. $V(w')$)
    -   If they didn’t get an offer, or **rejected** offer $w'$, they
        remain unemployed with values $U$

## Summarizing Value Functions

-   The value function for the unemployed worker is

    $$
    \begin{aligned}
    U &= c + \beta \left[ (1-\gamma) U + \gamma \mathbb{E}[\max\{U, V(w')\}]\right]\\
    &= c + \beta \left[ (1-\gamma) U + \gamma \sum_{i=1}^N \max\{U, V(w_i)\} p_i \right]
    \end{aligned}
    $$

-   The value function for the employed worker is

    $$
    V(w) = w + \beta \left[ (1-\alpha) V(w) + \alpha U \right]
    $$

-   Note since there are only $N$ values of $w_i$, we can define
    $V(w_i) \equiv V_i$ and solve for $N+1$ values (i.e.,
    $V_1, \ldots, V_N, U$)

## Fixed Point

-   Let
    $X \equiv \begin{bmatrix} V_1 & \ldots & V_N & U \end{bmatrix}^{\top}$

-   Define the Bellman Operator
    $T : \mathbb{R}^{N+1} \to \mathbb{R}^{N+1}$ stacking Bellman
    Equations $$
    T(X) \equiv \begin{bmatrix}
    w_1 + \beta \left[ (1-\alpha) V_1 + \alpha U \right] \\  
    \vdots \\
    w_N + \beta \left[ (1-\alpha) V_N + \alpha U \right] \\
    c + \beta \left[ (1-\gamma) U + \gamma \sum_{i=1}^N \max\{U, V_i\} p_i \right]
    \end{bmatrix}
    $$

-   Then the fixed point of $T(\cdot)$ (i.e., $T(X) = X$) is the
    solution to the problem